In [1]:
pip install pyodbc

Note: you may need to restart the kernel to use updated packages.


In [3]:
import csv
import pyodbc
import os
from datetime import datetime

def connect_to_db():
    conn = pyodbc.connect('DRIVER=ODBC Driver 17 for SQL Server;SERVER=Sagar\HUMBER_DB;DATABASE=PetRescueCharity;Trusted_Connection=yes;')
    return conn

def insert_address(cursor, address_data):
    cursor.execute("""
    INSERT INTO Address (unit_num, street_number, street_name, street_type, street_direction, postal_code, city, province)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    """, address_data)
    return cursor.execute("SELECT @@IDENTITY AS ID").fetchone()[0]

def insert_volunteer(cursor, volunteer_data):
    cursor.execute("""
    INSERT INTO Volunteer (first_name, last_name, group_leader)
    VALUES (?, ?, ?)
    """, volunteer_data)
    return cursor.execute("SELECT @@IDENTITY AS ID").fetchone()[0]

def insert_donation(cursor, donation_data):
    cursor.execute("""
    INSERT INTO Donation (donor_first_name, donor_last_name, donation_date, donation_amount, payment_method, address_id, volunteer_id)
    VALUES (?, ?, ?, ?, ?, ?, ?)
    """, donation_data)

from datetime import datetime

def process_csv(file_path, cursor):
    with open(file_path, 'r') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader)  # Skip header row
        for row in csvreader:
            address_data = (row[5], row[6], row[7], row[8], row[9], row[10], row[11], row[12])
            address_id = insert_address(cursor, address_data)

            volunteer_data = (row[13], row[14], row[15].upper() == 'TRUE')
            volunteer_id = insert_volunteer(cursor, volunteer_data)

            # Try both date formats
            try:
                donation_date = datetime.strptime(row[2], '%m/%d/%Y').strftime('%Y-%m-%d')
            except ValueError:
                try:
                    donation_date = datetime.strptime(row[2], '%Y-%m-%d').strftime('%Y-%m-%d')
                except ValueError:
                    print(f"Unable to parse date: {row[2]}. Skipping this row.")
                    continue

            donation_data = (row[0], row[1], donation_date, float(row[3]), row[4], address_id, volunteer_id)
            insert_donation(cursor, donation_data)
            
def main():
    conn = connect_to_db()
    cursor = conn.cursor()

    while True:
        file_path = input("Enter the path to the CSV file (or 'q' to quit): ")
        if file_path.lower() == 'q':
            break
          # Remove any surrounding quotes from the file path
        file_path = file_path.strip('"').strip("'")
        if not os.path.exists(file_path):
            print("File not found. Please enter a valid file path.")
            continue

        try:
            process_csv(file_path, cursor)
            conn.commit()
            print(f"Data from {file_path} has been successfully loaded into the database.")
        except Exception as e:
            conn.rollback()
            print(f"An error occurred while processing {file_path}: {str(e)}")

    conn.close()
    print("Database connection closed.")

if __name__ == "__main__":
    main()

Enter the path to the CSV file (or 'q' to quit): "C:\Users\sgrka\Downloads\Joaquin.csv"
Data from C:\Users\sgrka\Downloads\Joaquin.csv has been successfully loaded into the database.
Enter the path to the CSV file (or 'q' to quit): "C:\Users\sgrka\Downloads\Sagar.csv"
Data from C:\Users\sgrka\Downloads\Sagar.csv has been successfully loaded into the database.
Enter the path to the CSV file (or 'q' to quit): "C:\Users\sgrka\Downloads\Anmol.csv"
Data from C:\Users\sgrka\Downloads\Anmol.csv has been successfully loaded into the database.
Enter the path to the CSV file (or 'q' to quit): "C:\Users\sgrka\Downloads\Maleek.csv"
Data from C:\Users\sgrka\Downloads\Maleek.csv has been successfully loaded into the database.
Enter the path to the CSV file (or 'q' to quit): "C:\Users\sgrka\Downloads\Loan.csv"
Data from C:\Users\sgrka\Downloads\Loan.csv has been successfully loaded into the database.
Enter the path to the CSV file (or 'q' to quit): "C:\Users\sgrka\Downloads\Divya.csv"
Data from C:\Us